<a href="https://colab.research.google.com/github/mzwicker/ImplicitGeometricRegularization.pytorch/blob/master/ImplicitGeometricRegularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/mzwicker/ImplicitGeometricRegularization.pytorch
%cd ImplicitGeometricRegularization.pytorch

In [0]:
%run train_toy_2d.py